# Analyse des types de SST et clustering

**Objectif :** Analyser les différents types de sous-stations (SST) qui composent le réseau et les regrouper en clusters pour une meilleure compréhension et exploitation.

**Données sources :**
- **CAD** : `0_Data/0_Raw/RefFiles/Point_transmission_CAD_20260202.xlsx` — toutes les sous-stations CAD du réseau
- **Mesures archivées** : `0_Data/1_Structured/sst_unified.parquet` — DF avec colonnes `{U_NO_EGID}_TempRet` et `{U_NO_EGID}_PuisCpt`

**Colonnes analysées :**
- `U_TYPE_REGUL` : type de régulation
- `U_PROD_ECS` : production ECS (eau chaude sanitaire)
- `U_PUISSANCE` : puissance nominale (kW)

In [ ]:
# %% Imports et configuration
import re
from pathlib import Path
import sys
print(sys.executable)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Chemins relatifs au dossier 2_Program
PATH_CAD = Path("0_Data/0_Raw/RefFiles/Point_transmission_CAD_20260202.xlsx")
PATH_PARQUET = Path("0_Data/1_Structured/sst_unified.parquet")

# Style des graphiques
try:
    plt.style.use("seaborn-v0_8-whitegrid")
except OSError:
    plt.style.use("seaborn-whitegrid")
sns.set_palette("husl")

## 1. Chargement des données

- Charger le fichier Excel CAD
- Charger le parquet et extraire les EGIDs des colonnes (format `{EGID}_TempRet` ou `{EGID}_PuisCpt`)
- Filtrer le CAD pour ne garder que les SST ayant des mesures archivées

In [ ]:
# %% Chargement CAD
df_cad = pd.read_excel(PATH_CAD, sheet_name=0)
print(f"CAD : {len(df_cad)} sous-stations, {len(df_cad.columns)} colonnes")
print("Colonnes clés : U_NO_EGID, U_TYPE_REGUL, U_PROD_ECS, U_PUISSANCE")
df_cad.head(3)

In [ ]:
# %% Extraction des EGIDs du parquet (SST avec mesures archivées)
df_parquet = pd.read_parquet(PATH_PARQUET)
cols_measures = [c for c in df_parquet.columns if c not in ["timestamp_loc", "temp_ext_api"]]

egids_parquet = set()
for col in cols_measures:
    parts = col.split("_")
    if len(parts) >= 2 and parts[0].isdigit():
        egids_parquet.add(int(parts[0]))

print(f"Parquet : {len(df_parquet)} lignes, {len(egids_parquet)} SST avec mesures archivées")
print(f"Exemple EGIDs : {sorted(egids_parquet)[:10]}")

In [ ]:
# %% Fusion : filtrer CAD sur les SST avec mesures
df_cad["U_NO_EGID_num"] = pd.to_numeric(df_cad["U_NO_EGID"], errors="coerce")
df_sst = df_cad[df_cad["U_NO_EGID_num"].isin(egids_parquet)].copy()
df_sst = df_sst.drop(columns=["U_NO_EGID_num"])

print(f"SST analysées (CAD + mesures) : {len(df_sst)}")
df_sst[["U_NO_EGID", "U_TYPE_REGUL", "U_PROD_ECS", "U_PUISSANCE"]].head(10)

## 2. Analyse exploratoire des colonnes cibles

### 2.1 U_TYPE_REGUL (type de régulation)

In [ ]:
# %% U_TYPE_REGUL
vc_regul = df_sst["U_TYPE_REGUL"].value_counts(dropna=False)
print("Distribution U_TYPE_REGUL :")
print(vc_regul)
print(f"\nValeurs manquantes : {df_sst['U_TYPE_REGUL'].isna().sum()}")
print(f"Types distincts : {df_sst['U_TYPE_REGUL'].nunique()}")

fig, ax = plt.subplots(figsize=(12, 6))
vc_regul.head(20).plot(kind="barh", ax=ax)
ax.set_xlabel("Nombre de SST")
ax.set_title("Top 20 types de régulation (U_TYPE_REGUL)")
plt.tight_layout()
plt.show()

### 2.2 U_PROD_ECS (production ECS)

In [ ]:
# %% U_PROD_ECS
vc_ecs = df_sst["U_PROD_ECS"].value_counts(dropna=False)
print("Distribution U_PROD_ECS :")
print(vc_ecs)
print(f"\nValeurs manquantes : {df_sst['U_PROD_ECS'].isna().sum()}")

fig, ax = plt.subplots(figsize=(6, 4))
vc_ecs.plot(kind="bar", ax=ax)
ax.set_xticklabels([str(x) for x in vc_ecs.index], rotation=0)
ax.set_xlabel("U_PROD_ECS")
ax.set_ylabel("Nombre de SST")
ax.set_title("Production ECS (eau chaude sanitaire)")
plt.tight_layout()
plt.show()

### 2.3 U_PUISSANCE (puissance nominale en kW)

In [ ]:
# %% Parsing U_PUISSANCE (format "7kW", "180kW", etc.)
def parse_puissance(s):
    """Extrait la valeur numérique en kW depuis une chaîne comme '7kW' ou '180kW'."""
    if pd.isna(s):
        return np.nan
    m = re.search(r"([\d.]+)\s*k?W", str(s), re.I)
    return float(m.group(1)) if m else np.nan

df_sst["U_PUISSANCE_kW"] = df_sst["U_PUISSANCE"].apply(parse_puissance)
print("Statistiques U_PUISSANCE (kW) :")
print(df_sst["U_PUISSANCE_kW"].describe())
print(f"\nValeurs manquantes : {df_sst['U_PUISSANCE_kW'].isna().sum()}")

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
df_sst["U_PUISSANCE_kW"].hist(bins=30, ax=axes[0], edgecolor="black", alpha=0.7)
axes[0].set_xlabel("Puissance (kW)")
axes[0].set_ylabel("Nombre de SST")
axes[0].set_title("Distribution de la puissance")

df_sst["U_PUISSANCE_kW"].plot(kind="box", ax=axes[1])
axes[1].set_ylabel("Puissance (kW)")
axes[1].set_title("Boîte à moustaches")
plt.tight_layout()
plt.show()

## 3. Préparation des features pour le clustering

Encodage des variables catégorielles et gestion des valeurs manquantes.

In [ ]:
# %% Préparation du DataFrame de clustering
df_cluster = df_sst[["U_NO_EGID", "U_TYPE_REGUL", "U_PROD_ECS", "U_PUISSANCE_kW"]].copy()

# Remplacer NaN par une catégorie pour U_TYPE_REGUL et U_PROD_ECS
df_cluster["U_TYPE_REGUL"] = df_cluster["U_TYPE_REGUL"].fillna("_MANQUANT_")
df_cluster["U_PROD_ECS"] = df_cluster["U_PROD_ECS"].fillna(-1).astype(str)  # -1 = manquant

# Encodage U_TYPE_REGUL (LabelEncoder pour KMeans)
le_regul = LabelEncoder()
df_cluster["U_TYPE_REGUL_enc"] = le_regul.fit_transform(df_cluster["U_TYPE_REGUL"].astype(str))

# Encodage U_PROD_ECS (0, 1, -1 pour manquant)
def map_ecs(x):
    s = str(x).strip()
    if s in ("0", "0.0"): return 0
    if s in ("1", "1.0"): return 1
    return -1
df_cluster["U_PROD_ECS_enc"] = df_cluster["U_PROD_ECS"].apply(map_ecs)

# U_PUISSANCE_kW : remplir les manquants par la médiane
med_puissance = df_cluster["U_PUISSANCE_kW"].median()
df_cluster["U_PUISSANCE_kW_fill"] = df_cluster["U_PUISSANCE_kW"].fillna(med_puissance)

# Features pour clustering
features = ["U_TYPE_REGUL_enc", "U_PROD_ECS_enc", "U_PUISSANCE_kW_fill"]
X = df_cluster[features].values

# Normalisation (important pour KMeans)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Features préparées :", features)
print("Shape X_scaled :", X_scaled.shape)
print("Exemple (5 premières lignes) :")
pd.DataFrame(X_scaled[:5], columns=features)

## 4. Clustering KMeans

Choix du nombre de clusters via le score de silhouette.

In [ ]:
# %% Choix du nombre de clusters (silhouette)
k_range = range(2, 15)
silhouette_scores = []

for k in k_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    silhouette_scores.append(score)

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(list(k_range), silhouette_scores, "o-", linewidth=2, markersize=8)
ax.set_xlabel("Nombre de clusters (k)")
ax.set_ylabel("Score de silhouette")
ax.set_title("Choix du nombre de clusters")
ax.set_xticks(list(k_range))
plt.tight_layout()
plt.show()

best_k = list(k_range)[np.argmax(silhouette_scores)]
print(f"Meilleur k (silhouette max) : {best_k}")

In [ ]:
# %% Clustering final avec le meilleur k (ou k=6 par défaut)
n_clusters = best_k  # Modifier si besoin (ex. k=6 pour interprétabilité)
km_final = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
df_cluster["cluster"] = km_final.fit_predict(X_scaled)

print(f"Répartition des {n_clusters} clusters :")
print(df_cluster["cluster"].value_counts().sort_index())

## 5. Analyse des clusters

Profil de chaque cluster selon U_TYPE_REGUL, U_PROD_ECS et U_PUISSANCE.

In [ ]:
# %% Profil des clusters
for c in range(n_clusters):
    subset = df_cluster[df_cluster["cluster"] == c]
    print(f"\n--- Cluster {c} ({len(subset)} SST) ---")
    print("  U_TYPE_REGUL (top 3) :", subset["U_TYPE_REGUL"].value_counts().head(3).to_dict())
    print("  U_PROD_ECS :", subset["U_PROD_ECS"].value_counts().to_dict())
    print("  U_PUISSANCE_kW : min={:.0f}, max={:.0f}, médiane={:.0f}".format(
        subset["U_PUISSANCE_kW"].min(), subset["U_PUISSANCE_kW"].max(), subset["U_PUISSANCE_kW"].median()
    ))

In [ ]:
# %% Visualisation 2D (PCA pour réduire en 2D si > 2 features)
from sklearn.decomposition import PCA

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

df_viz = pd.DataFrame({
    "PC1": X_pca[:, 0],
    "PC2": X_pca[:, 1],
    "cluster": df_cluster["cluster"],
    "U_PUISSANCE_kW": df_cluster["U_PUISSANCE_kW_fill"],
    "U_TYPE_REGUL": df_cluster["U_TYPE_REGUL"],
})

fig, ax = plt.subplots(figsize=(10, 8))
sizes = np.clip(df_viz["U_PUISSANCE_kW"] / 5, 20, 200)
scatter = ax.scatter(
    df_viz["PC1"], df_viz["PC2"],
    c=df_viz["cluster"], cmap="tab10", alpha=0.6, s=sizes
)
ax.set_xlabel("Composante principale 1")
ax.set_ylabel("Composante principale 2")
ax.set_title("Clusters SST (taille = puissance kW)")
plt.colorbar(scatter, ax=ax, label="Cluster")
plt.tight_layout()
plt.show()

In [ ]:
# %% Heatmap : répartition U_TYPE_REGUL par cluster
cross_regul = pd.crosstab(df_cluster["cluster"], df_cluster["U_TYPE_REGUL"])
# Garder les types les plus fréquents pour la lisibilité
top_types = [c for c in df_cluster["U_TYPE_REGUL"].value_counts().head(12).index if c in cross_regul.columns]
cross_regul_top = cross_regul[top_types] if top_types else cross_regul.iloc[:, :12]

fig, ax = plt.subplots(figsize=(14, 6))
sns.heatmap(cross_regul_top.T, annot=True, fmt="d", cmap="YlOrRd", ax=ax)
ax.set_xlabel("Cluster")
ax.set_ylabel("Type de régulation")
ax.set_title("Répartition U_TYPE_REGUL par cluster")
plt.tight_layout()
plt.show()

## 6. Export des résultats

Export du DataFrame enrichi avec les clusters pour réutilisation.

In [ ]:
# %% Export
out_path = Path("0_Data/1_Structured/sst_avec_clusters.parquet")
df_export = df_sst.merge(
    df_cluster[["U_NO_EGID", "cluster"]],
    on="U_NO_EGID",
    how="inner"
)
df_export.to_parquet(out_path, index=False)
print(f"Exporté : {out_path} ({len(df_export)} lignes)")
df_export[["U_NO_EGID", "U_TYPE_REGUL", "U_PROD_ECS", "U_PUISSANCE", "cluster"]].head(10)